In [1]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName('key-value_rdd_op_joins')
sc = SparkContext(conf = conf)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/05 04:07:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


 # Operaions
 - groupByKey
     - KeyValueRDD.groupByKey()
     - 그룹핑 후에 특정 Transformations 같은 연산
     - key 값이 있는 상태에서 시작
 - 

In [2]:
rdd = sc.parallelize([
                     ('짜장면', 15),
                     ('짬뽕',10),
                     ('짜장면', 5)
                     ])

In [3]:
g_rdd = rdd.groupByKey()
g_rdd.collect()

[('짜장면', <pyspark.resultiterable.ResultIterable at 0x7f28d4027610>),
 ('짬뽕', <pyspark.resultiterable.ResultIterable at 0x7f28d4027d90>)]

In [4]:
g_rdd.mapValues(len).collect()

[('짜장면', 2), ('짬뽕', 1)]

In [6]:
g_rdd.mapValues(list).collect()

[('짜장면', [15, 5]), ('짬뽕', [10])]

In [11]:
# groupBy 사용하기
rdd = sc.parallelize([
    'C', 'C++','C#', 'Python', 'Java', "JavaScript"
])

In [12]:
# groupBy는 그룹핑할 키에 대한 정의를 개발자가 직접한다.
grouped = rdd.groupBy(lambda x : x[0])
grouped.glom().collect()

[[('C', <pyspark.resultiterable.ResultIterable at 0x7f28bcd87400>),
  ('P', <pyspark.resultiterable.ResultIterable at 0x7f28e4610640>),
  ('J', <pyspark.resultiterable.ResultIterable at 0x7f28bcd00430>)]]

In [13]:
grouped.mapValues(list).collect()

[('C', ['C', 'C++', 'C#']), ('P', ['Python']), ('J', ['Java', 'JavaScript'])]

In [14]:
grouped.getNumPartitions()

1

In [15]:
g_rdd.getNumPartitions()

1

In [16]:
# groupByKey는 K-V RDD를 사용할 때 Key가 알아서 그룹핑의 기준이 된다.

x = sc.parallelize([
    ("MATH", 7), ("MATH", 2), ("ENGLISH", 7),
    ("SCIENCE", 7), ("ENGLISH", 4), ("ENGLISH", 9),
    ("MATH", 8), ("MATH", 3), ("ENGLISH", 4),
    ("SCIENCE", 6), ("SCIENCE", 9), ("SCIENCE", 5)
], 3)

y = x.groupByKey()

In [17]:
y.collect()

[('MATH', <pyspark.resultiterable.ResultIterable at 0x7f28bccefa60>),
 ('ENGLISH', <pyspark.resultiterable.ResultIterable at 0x7f28bcbc7be0>),
 ('SCIENCE', <pyspark.resultiterable.ResultIterable at 0x7f28bcbc7d30>)]

In [18]:
y.glom().collect()

[[('MATH', <pyspark.resultiterable.ResultIterable at 0x7f28bce0c160>),
  ('ENGLISH', <pyspark.resultiterable.ResultIterable at 0x7f28bc9f9fa0>),
  ('SCIENCE', <pyspark.resultiterable.ResultIterable at 0x7f28bcd271c0>)],
 [],
 []]

# Joins

In [19]:
# Inner Join : 서로간에 존재하는 키만 합쳐준다
rdd1 = sc.parallelize([
    ('foo', 1),
    ('goo', 2),
    ('hoo', 3)
])

rdd2 = sc.parallelize([
    ('foo', 1),
    ('goo', 2),
    ('goo', 10),
    ('moo', 6)
])
rdd1.join(rdd2).collect()

[('foo', (1, 1)), ('goo', (2, 2)), ('goo', (2, 10))]

Outer Join

In [20]:
rdd1.leftOuterJoin(rdd2).collect()

[('foo', (1, 1)), ('goo', (2, 2)), ('goo', (2, 10)), ('hoo', (3, None))]

In [21]:
rdd1.rightOuterJoin(rdd2).collect()

[('foo', (1, 1)), ('moo', (None, 6)), ('goo', (2, 2)), ('goo', (2, 10))]

In [22]:
sc.stop()